In [ ]:
!pip -q install kaggle

In [ ]:
import os
import json
from google.colab import userdata

# 1. Récupération sécurisée des identifiants (Assurez-vous d'avoir créé les Secrets dans Colab)
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# 2. Définition du chemin et création du répertoire
kaggle_path = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_path, exist_ok=True)

# 3. Création du fichier kaggle.json [cite: 31]
with open(os.path.join(kaggle_path, "kaggle.json"), "w") as f:
    json.dump({
        "username": os.environ['KAGGLE_USERNAME'],
        "key": os.environ['KAGGLE_KEY']
    }, f)

# 4. Sécurisation du fichier [cite: 32]
!chmod 600 ~/.kaggle/kaggle.json

# 5. Téléchargement du dataset [cite: 34]
!kaggle datasets download -d chetankv/dogs-cats-images -p data/ --unzip

# 6. Vérification du contenu téléchargé
print("\nDossiers téléchargés :")
!ls data/dog\ vs\ cat/dataset

In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random
import numpy as np


SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    device = "cuda"
else:
    device = "cpu"


# 1. Définition des transformations (Point crucial pour l'amélioration) [cite: 56]
data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)), # Taille standard pour le modèle de base [cite: 53]
    transforms.ToTensor(),
])

# 2. Chargement via ImageFolder [cite: 17, 41]
train_dir = "data/dog vs cat/dataset/training_set"
test_dir = "data/dog vs cat/dataset/test_set"

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

print(f"Dataset prêt ! Classes trouvées : {train_data.classes}")
print(f"Nombre de donnée train : {len(train_data)}")
print(f"Nombre de donnée test : {len(test_data)}")

In [ ]:
class SimpleCNN_AvgPool(nn.Module):
    def __init__(self):
        super(SimpleCNN_AvgPool, self).__init__()
        # Couche 1
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.AvgPool2d(kernel_size=2) # On prend la moyenne au lieu du max

        # Couche 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.AvgPool2d(kernel_size=2) # On prend la moyenne

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 16 * 16, 2)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.flatten(x)
        return self.fc(x)

model_base = SimpleCNN_AvgPool().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_base.parameters(), lr=0.01)

In [ ]:
import time
# 1. Configuration de l'entraînement
epochs = 10
train_losses = []
test_accuracies = []

print(f"Début de l'entraînement sur {device}...")

for epoch in range(epochs):
    start_time = time.time()

    # --- PHASE D'ENTRAÎNEMENT ---
    model_base.train()
    train_loss = 0

    for batch, (X, y) in enumerate(train_loader):
        # Envoi des données sur le GPU/CPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model_base(X)

        # 2. Calcul de la perte
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward (Backpropagation)
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # --- PHASE DE TEST (ÉVALUATION) ---
    model_base.eval()
    test_acc = 0
    with torch.inference_mode(): # Mode économie pour le test
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_pred_logits = model_base(X_test)

            # Calcul de la précision
            test_pred_labels = torch.argmax(test_pred_logits, dim=1)
            test_acc += (test_pred_labels == y_test).sum().item()

    # Calcul des moyennes pour l'affichage
    train_loss /= len(train_loader)
    test_acc /= len(test_data)

    train_losses.append(train_loss)
    test_accuracies.append(test_acc)

    end_time = time.time()
    print(f"Époque: {epoch+1} | Perte: {train_loss:.4f} | Précision Test: {test_acc*100:.2f}% | Temps: {end_time-start_time:.1f}s")

print("\nEntraînement terminé !")

In [ ]:
def predict_and_plot(model, dataset, n_images=5):
    model.eval()
    plt.figure(figsize=(15, 5))
    for i in range(n_images):
        # On prend une image au hasard dans le dataset de test
        idx = torch.randint(0, len(dataset), (1,)).item()
        image, label = dataset[idx]

        # Prédiction
        with torch.inference_mode():
            pred_logits = model(image.unsqueeze(0).to(device))
            pred_label = torch.argmax(pred_logits, dim=1).item()

        # Affichage
        plt.subplot(1, n_images, i+1)
        plt.imshow(image.permute(1, 2, 0))
        color = "green" if pred_label == label else "red"
        plt.title(f"Vrai: {dataset.classes[label]}\nPred: {dataset.classes[pred_label]}", color=color)
        plt.axis("off")
    plt.show()

predict_and_plot(model_base, test_data)

In [ ]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Couche 1 : Détecte les formes de base (bords, lignes)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2) # Divise la taille de l'image par 2

        # Couche 2 : Détecte des formes plus complexes
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Couche de sortie : Classifie en 2 catégories (Chat ou Chien)
        self.flatten = nn.Flatten()
        # 16*16 est la taille de l'image après deux passages en MaxPool (64/2/2 = 16)
        self.fc = nn.Linear(32 * 16 * 16, 2)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.flatten(x)
        return self.fc(x)

# Instanciation et envoi sur le GPU/CPU
model_base = SimpleCNN().to(device)
# print(model_base)

In [ ]:
#experience 1
model_base = SimpleCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model_base.parameters(), lr=0.0001)



In [ ]:
epochs = 10
train_losses_adam = []
test_accuracies_adam = []

for epoch in range(epochs):
    model_base.train()
    train_loss = 0
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        y_pred = model_base(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Phase de Test
    model_base.eval()
    test_acc = 0
    with torch.inference_mode():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_pred_logits = model_base(X_test)
            test_acc += (torch.argmax(test_pred_logits, dim=1) == y_test).sum().item()

    train_loss /= len(train_loader)
    test_acc /= len(test_data)

    train_losses_adam.append(train_loss)
    test_accuracies_adam.append(test_acc)

    print(f"Époque: {epoch+1} | Perte: {train_loss:.4f} | Précision Test: {test_acc*100:.2f}%")

In [ ]:
# --- EXPÉRIENCE 2 : DATA AUGMENTATION ---

# 1. Transformations d'entraînement (avec augmentation)
train_transform_aug = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),      # Retourne l'image aléatoirement
    transforms.RandomRotation(degrees=15),       # Rotation légère
    transforms.ColorJitter(brightness=0.2),      # Change un peu la luminosité
    transforms.ToTensor(),
])

# 2. Transformations de test (restent simples)
test_transform_simple = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor(),
])

# 3. Chargement des données avec les nouveaux transforms
# On utilise les chemins définis dans ton notebook
train_dir = "data/dog vs cat/dataset/training_set"
test_dir = "data/dog vs cat/dataset/test_set"

train_data_aug = datasets.ImageFolder(root=train_dir, transform=train_transform_aug)
test_data_simple = datasets.ImageFolder(root=test_dir, transform=test_transform_simple)

# 4. Création des nouveaux DataLoaders
train_loader_aug = DataLoader(dataset=train_data_aug, batch_size=32, shuffle=True)
test_loader_simple = DataLoader(dataset=test_data_simple, batch_size=32, shuffle=False)

print(f"Data Augmentation activée pour {len(train_data_aug)} images d'entraînement.")

In [ ]:

# On réinitialise le modèle et l'optimiseur
model_aug = SimpleCNN().to(device)
optimizer_aug = torch.optim.Adam(params=model_aug.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

epochs = 10
train_losses_aug = []
test_accuracies_aug = []

print("Début de l'entraînement avec Data Augmentation...")

for epoch in range(epochs):
    model_aug.train()
    train_loss = 0
    for X, y in train_loader_aug: # Utilisation du loader augmenté
        X, y = X.to(device), y.to(device)
        y_pred = model_aug(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer_aug.zero_grad()
        loss.backward()
        optimizer_aug.step()

    # Évaluation sur le set de test (non augmenté)
    model_aug.eval()
    test_acc = 0
    with torch.inference_mode():
        for X_test, y_test in test_loader_simple:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_pred = model_aug(X_test)
            test_acc += (torch.argmax(test_pred, dim=1) == y_test).sum().item()

    train_loss /= len(train_loader_aug)
    test_acc /= len(test_data_simple)

    train_losses_aug.append(train_loss)
    test_accuracies_aug.append(test_acc)

    print(f"Époque: {epoch+1} | Perte: {train_loss:.4f} | Précision Test: {test_acc*100:.2f}%")

In [ ]:
# --- EXPÉRIENCE 3 : AJOUT D'UNE 3ème COUCHE DE CONVOLUTION ---


class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        # Couche 1 : 64x64 -> 32x32
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # Couche 2 : 32x32 -> 16x16
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # Couche 3 (Nouvelle) : 16x16 -> 8x8
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.flatten = nn.Flatten()
        # Le calcul change : 128 filtres * image de 8x8 pixels = 8192 entrées
        self.fc = nn.Linear(128 * 8 * 8, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.flatten(x)
        return self.fc(x)

# Initialisation du modèle profond
model_base = DeepCNN().to(device)
# On garde l'optimiseur Adam qui a très bien fonctionné
optimizer= torch.optim.Adam(params=model_base.parameters(), lr=0.0001)

In [ ]:
epochs = 10
train_losses_adam = []
test_accuracies_adam = []

for epoch in range(epochs):
    model_base.train()
    train_loss = 0
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        y_pred = model_base(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Phase de Test
    model_base.eval()
    test_acc = 0
    with torch.inference_mode():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_pred_logits = model_base(X_test)
            test_acc += (torch.argmax(test_pred_logits, dim=1) == y_test).sum().item()

    train_loss /= len(train_loader)
    test_acc /= len(test_data)

    train_losses_adam.append(train_loss)
    test_accuracies_adam.append(test_acc)

    print(f"Époque: {epoch+1} | Perte: {train_loss:.4f} | Précision Test: {test_acc*100:.2f}%")